In [1]:




from jupyter_dash import JupyterDash

import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output

import plotly.express as px
import pandas as pd
import base64
import os

from AnimalShelter import AnimalShelter



# --- Mongo connection ---
a = AnimalShelter(username="aacuser", password="aacuser",
                  host="nv-desktop-services.apporto.com", port=30685)

# --- Rescue rules (from the spec) ---
BREEDS = {
    "WATER":    ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"],
    "MOUNTAIN": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"],
    "DISASTER": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"],
}
SEX =  {"WATER":"Intact Female", "MOUNTAIN":"Intact Male", "DISASTER":"Intact Male"}
AGE =  {"WATER":(26,156), "MOUNTAIN":(26,156), "DISASTER":(20,300)}

def make_query(key: str) -> dict:
    if key == "RESET":
        return {}
    lo, hi = AGE[key]
    return {
        "animal_type": "Dog",
        "breed": {"$in": BREEDS[key]},
        "sex_upon_outcome": SEX[key],
        "age_upon_outcome_in_weeks": {"$gte": lo, "$lte": hi},
    }

def to_df(query=None):
    rows = a.read(query or {})
    df = pd.DataFrame(rows)
    for c in ("location_lat", "location_long", "age_upon_outcome_in_weeks"):
        if c in df.columns: df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

# Logo (put grazioso_logo.png next to the notebook)
def embed_logo(path="grazioso_logo.png"):
    try:
        with open(path,"rb") as f:
            return "data:image/png;base64," + base64.b64encode(f.read()).decode()
    except:
        return None

LOGO = embed_logo("Grazioso Salvare Logo.png")

app = JupyterDash(__name__)

app.layout = html.Div([
    # Header
    html.Div([
        html.Img(src=LOGO, style={"height":"70px"}) if LOGO else html.Div("GRAZIOSO"),
        html.H1("Grazioso Salvare | Rescue Dog Finder — Mark Harper", style={"marginLeft":"10px"})
    ], style={"display":"flex","alignItems":"center","gap":"12px","marginBottom":"10px"}),

    # Filters
    html.Div([
        dcc.RadioItems(
            id="rescue_filter",
            options=[
                {"label":"Water Rescue", "value":"WATER"},
                {"label":"Mountain/Wilderness Rescue", "value":"MOUNTAIN"},
                {"label":"Disaster/Individual Tracking", "value":"DISASTER"},
                {"label":"Reset", "value":"RESET"},
            ],
            value="RESET",
            inline=True
        )
    ], style={"background":"#f6f6f6","padding":"10px","borderRadius":"8px","marginBottom":"12px"}),

    # Data table
    dash_table.DataTable(
        id="tbl",
        columns=[{"name":c, "id":c} for c in [
            "animal_id","name","breed","animal_type","sex_upon_outcome",
            "age_upon_outcome_in_weeks","outcome_type","location_lat","location_long"
        ]],
        data=[],
        page_size=10,
        sort_action="native",
        filter_action="native",
        style_table={"overflowX":"auto"},
        style_cell={"textAlign":"left","padding":"6px"},
    ),

    # Map + second chart
    html.Div([
        dl.Map(center=[30.27,-97.74], zoom=9, style={"height":"450px","width":"100%"},
               children=[dl.TileLayer(), dl.LayerGroup(id="markers")]),
        dcc.Graph(id="chart2")
    ])
])
@app.callback(
    Output("tbl","data"),
    Output("markers","children"),
    Output("chart2","figure"),
    Input("rescue_filter","value")
)
def update(choice):
    try:
        query = {} if choice == "RESET" else make_query(choice)
        df = to_df(query)

        # ---- sanitize for the DataTable (NaN -> None so JSON is valid) ----
        df_disp = df.copy()
        df_disp = df_disp.where(pd.notnull(df_disp), None)

        # ---- Leaflet markers ----
        markers = []
        if not df.empty:
            df2 = df.dropna(subset=["location_lat", "location_long"])
            for _, r in df2.iterrows():
                lat = float(r["location_lat"]); lon = float(r["location_long"])
                markers.append(
                    dl.Marker(
                        position=[lat, lon],
                        children=[
                            dl.Tooltip(r.get("breed","")),
                            dl.Popup([html.H4(r.get("name","")), html.P(r.get("breed",""))])
                        ]
                    )
                )

        # ---- Top Breeds bar (Top 10) ----
        if not df.empty and df["breed"].notna().any():
            top = (df["breed"].value_counts()
                   .reset_index()
                   .rename(columns={"index":"breed","breed":"count"})
                   .head(10))
            fig2 = px.bar(top, x="breed", y="count", title="Top Breeds (Top 10)")
        else:
            fig2 = px.bar(title="Top Breeds (no data)")

        return df_disp.to_dict("records"), markers, fig2

    except Exception as e:
        print("callback error:", e)
        # safe fallback so the app doesn't break
        return [], [], px.bar(title="Top Breeds (error)")
import logging, sys
logging.getLogger("werkzeug").setLevel(logging.ERROR)
app.run_server(mode="inline", debug=False)